<h2>US Stock Market Capitalization to GDP (Buffet Indicator)</h2>

<p style="font-size:15px;">by Lewin Hafner</p>
<p style="font-size:15px;">Latest Update: October 4th, 2020</p>

<h3>The Ratio of the Two</h3>
<p>Given that the stock market represents primarily expectations of future economic activity, and the GDP
    is a measure of most recent economic activity, the ratio of these two data series represents expected
    future growth relative to current performance. This is similar in nature to how we think about the
    PE ratio of a particular stock. It stands to reason that this ratio would remain relatively stable
    over time, and increase slowly over time as technology allows for the same labor and capital to be
    used ever more efficiently.</p>

<h3>1. Setup</h3>
<p>Imports all necessary libraries.</p>

In [1]:
import pandas_datareader as pdr
import pandas as pd
import datetime
import plotly.express as px
import matplotlib.pyplot as plt

<h3>2. FRED Data and Calculation</h3>
<p>The indicator is calculated as follows:</p>
<ul>
<li>Pull quarterly GDP data from FRED</li>
<li>Pull the Wilshire 5000 Full Cap Price Index from FRED</li>
<li>Convert daily series to quarterly series</li>
<li>Calculate the index as [Quarterly Market Cap] / [GDP]</li>
</ul>

In [6]:
%%capture
# Defines start and end in yyyy, mm, dd format
start = datetime.datetime (1971, 1, 1)
end = datetime.datetime (2020, 12, 31)

# Get Market Cap Data with FRED ID#
data_series = "WILL5000PRFC"
market_cap = pdr.DataReader(data_series, 'fred', start, end)

# Drop NaN values from market_cap dataframe
market_cap = market_cap.dropna()

# Convert dataframe index to datetime object
market_cap.index = pd.to_datetime(market_cap.index)

# Get GDP Data with FRED ID#
data_series = "GDP"
gdp = pdr.DataReader(data_series, 'fred', start, end)

# Drop NaN values from gdp dataframe
gdp = gdp.dropna()

# Convert dataframe index to datetime object
gdp.index = pd.to_datetime(gdp.index)

# Create quarterly market_cap
market_cap_quarterly = pd.DataFrame()

market_cap_quarterly = market_cap
market_cap_quarterly.resample("3M").last()

# Merge the two dataframes using pandas' merge() on their indices
market_cap_to_gdp = pd.merge(market_cap_quarterly, gdp, left_index=True, right_index=True)

# Create Date column (x-label)#
market_cap_to_gdp["Date"] = market_cap_to_gdp.index

# Multiply the Wilshire 5000 Full Cap index by $1.19 billion (1980 Dollars)
market_cap_to_gdp["WILL5000PRFC"] = market_cap_to_gdp["WILL5000PRFC"] * 1190000000

# Calculate the indicator as market capitalization / GDP
market_cap_to_gdp["Buffett Indicator"] = market_cap_to_gdp["WILL5000PRFC"] / market_cap_to_gdp["GDP"]

<h3>3. Plotting and styling</h3>
<p>Plots the final ratio as an area chart using Plotly Express</p>

In [7]:
indicator_chart = px.area(market_cap_to_gdp, x='Date', y='Buffett Indicator',
    title='US Stock Market Capitalization to GDP (The Buffett Indicator)',
    color_discrete_sequence=["#004a4a "],
    labels="",
    width=950,
    height=600)

indicator_chart.update_layout(margin=dict(l=100,r=40,t=80, b=40),
    plot_bgcolor = "#eeeeee",
    paper_bgcolor="#f8f8f8") 

indicator_chart.show()